In [2]:
import requests
from dotenv import load_dotenv
import os
import mysql.connector
import json
import csv
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from constants import DB_DRIVER_NAMES, YEARS, TYRE_PARAMS
import driver

In [3]:
load_dotenv()

db_host = os.getenv('DB_HOST')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_name = os.getenv('DB_NAME')

In [4]:
quali_df = pd.read_csv('all_quali_data.csv')
race_df = pd.read_csv('all_race_data.csv')
print(quali_df)

      year  race_name            driver  Qualifying_time
0     2022    bahrain   charles-leclerc           90.558
1     2022    bahrain    max-verstappen           90.681
2     2022    bahrain   carlos-sainz-jr           90.687
3     2022    bahrain      sergio-perez           90.921
4     2022    bahrain    lewis-hamilton           91.238
...    ...        ...               ...              ...
1281  2024  sao-paulo    lewis-hamilton           91.150
1282  2024  sao-paulo    oliver-bearman           91.229
1283  2024  sao-paulo  franco-colapinto           91.270
1284  2024  sao-paulo   nico-hulkenberg           91.623
1285  2024  sao-paulo       guanyu-zhou           92.263

[1286 rows x 4 columns]


In [5]:
#Each lap we call this method to remove X amount of fuel for the lap. This X will change depending on the track.
#For now it is Dutch GP which burns 1.49 kg fuel per lap
def get_fuel_for_lap(current_fuel):
    current_fuel -= 1.49
    return current_fuel

In [6]:
#fuel level must be in kilograms
def fuel_burn_affect(fuel_level):
    return fuel_level * 0.03

In [7]:
#This method returns the average pitstop loss for a given race. At the moment it is just the Dutch GP
#More will be added later
def get_avg_pit_loss():
    return float(21.355)

In [8]:
#Here we retrieve the base lap time which is equivalent to the qualifying time
def get_base_time(race, year, driver):
    base_time = (quali_df[(quali_df.year == year) & (quali_df.race_name == f"{race}") & (quali_df.driver == f"{driver}")])
    return float(base_time.iloc[0, 3])

In [9]:
#This is the function for the basic tyre affect.
def get_tyre_affect(lap_number, tyre_compound, race_name):
    tyre_affect = 0
    params = TYRE_PARAMS[race_name][tyre_compound]
    tyre_affect = params["base"] + params["linear"] * lap_number + params["quadratic"] * (lap_number ** 2)
    return tyre_affect

Choosing when to pit will be crucial in determining realism. The initial way we will do this is to see how much the time has increased over the stint. This will be done by checking the quickest lap time within the first 5 laps and then waiting until the tyre reaches a certain threshold above that.

In [10]:
#This method is to check weather a pit stop is now a smart option
#For the basic implementation we will just assume that if the tyre has started to drop off then a pit stop is needed
def time_to_pit(tyre_compound, num_laps, stint_lap, lap_number, lap_time, lap_times):

    remaining_laps = num_laps - lap_number

    #print(f"remaining laps: {remaining_laps} ")
    if remaining_laps < 0:
        return True
    elif remaining_laps < 10 or stint_lap < 6:

        return False

    elif tyre_compound == "soft":

        for i in range(1, 3):

            if(lap_times[i]+ 1 < lap_time):

                return True
            
        return False
    
    elif tyre_compound == "medium":

        for i in range(1, 3):

            if(lap_times[i]+ 1.9 < lap_time):

                return True
            
        return False
    
    else:

        for i in range(1, 3):

            if(lap_times[i]+ 2.3 < lap_time):

                return True
            
        return False

This next function will actually simulate the pitstop and change the tyre compound. Later down the line it will also calculate positionally where the driver will come out etc. 

In [11]:
def simulate_pit_stop(current_compound, num_laps, lap_number):
    laps_remaining = num_laps - lap_number
    if current_compound == "soft":
        if laps_remaining > 25:
            next_compound = "hard"
        else:
            next_compound = "medium"
    elif current_compound == "medium":
        if laps_remaining < 13:
            next_compound = "soft"
        else:
            next_compound = "hard"
    else:
        if laps_remaining < 13:
            next_compound = "soft"
        else:
            next_compound = "medium"
    return next_compound

In [12]:
def simulate_lap(fuel_level, race, year, driver,  lap_number, compound, base_lap_time):

    
    fuel_affect = fuel_burn_affect(fuel_level)
    tyre_affect = get_tyre_affect(lap_number, compound, race)
    
    lap_time = base_lap_time + fuel_affect + tyre_affect

    return lap_time


In [13]:
def simulate_stint(fuel_level, num_laps, lap_number, compound, base_lap_time, race, year, driver):

    stint_lap = 1
    current_fuel = fuel_level
    lap_times = []
    current_fuel = get_fuel_for_lap(current_fuel)
    lap_time = simulate_lap(current_fuel, race, year, driver, stint_lap, compound, base_lap_time)

    lap_times.append(lap_time)

    while(time_to_pit(compound, num_laps, stint_lap, lap_number, lap_time, lap_times) == False):
        current_fuel = get_fuel_for_lap(current_fuel)
        
        stint_lap += 1

        lap_number += 1

        #print(f"lap number: {lap_number}")

        lap_time = simulate_lap(current_fuel, race, year, driver, stint_lap, compound, base_lap_time)

        #print(f"lap time: {lap_time}")
        
        lap_times.append(lap_time)
        

    next_compound = simulate_pit_stop(compound, num_laps, lap_number)

    print(f"pitted on lap: {lap_number} onto compound: {next_compound}")

    lap_number += 1
    
    return lap_times, lap_number, next_compound, current_fuel

In [14]:
def simulate_race(num_laps, starting_compound, fuel_level, race, year, driver):

    compound = starting_compound

    total_lap_times = []

    lap = 1

    base_lap_time = get_base_time(race, year, driver)

    current_fuel = get_fuel_for_lap(fuel_level)

    while(len(total_lap_times) < num_laps):

        stint_lap_times, lap, next_compound, current_fuel = simulate_stint(current_fuel, num_laps, lap, compound, base_lap_time, race, year, driver)
        compound = next_compound
        for i in range(len(stint_lap_times)):

            total_lap_times.append(stint_lap_times[i])
           
        
    

    return total_lap_times[:num_laps]


This is the basic simulation. One driver, one race. The dutch grand prix is useful because it is a fairly standard race. One stop and a medium level of tyre degredation. However, we can still see differences in the strategy as if we start on the mediums our race time becomes around 12 seconds quicker in total. That may not seem a lot over a 71 lap race, however it is as in this race 12 seconds was the gap between 2nd and 5th

In [15]:
#Netherlands Test
lap_times_medium = simulate_race(71, "medium", 110, "netherlands", 2024, "lando-norris")
total_time_medium = 0 
total_time_hard = 0
prev_lap = 0
changes_med = []
for i in range(len(lap_times_medium)):
    lap_change = prev_lap - lap_times_medium[i]
    if (-12 < lap_change < 7):
        changes_med.append(lap_change)
    prev_lap = lap_times_medium[i]
    total_time_medium = total_time_medium + lap_times_medium[i]
lap_times_hard = simulate_race(71, "hard", 110, "netherlands", 2024, "lando-norris")
for i in range(len(lap_times_hard)):
    total_time_hard = total_time_hard + lap_times_hard[i]
print(f"Total time for starting on the medium: {total_time_medium}")
print(f"Total time for starting on the hard: {total_time_hard}")
avg_time_change_med = sum(changes_med) / (len(changes_med)+1)
print(f"Average lap time change for medium: {avg_time_change_med}")

pitted on lap: 28 onto compound: hard
pitted on lap: 72 onto compound: soft
pitted on lap: 47 onto compound: medium
pitted on lap: 72 onto compound: soft
Total time for starting on the medium: 5519.1377
Total time for starting on the hard: 5531.8301
Average lap time change for medium: -0.03471267605633809


In [16]:
#Japan Test
lap_times_medium = simulate_race(71, "medium", 110, "netherlands", 2024, "lando-norris")
for i in range(len(lap_times_medium)):
    print(lap_times_medium[i])
print(sum(lap_times_medium))

pitted on lap: 28 onto compound: hard
pitted on lap: 72 onto compound: soft
76.8891
76.8579
76.8347
76.8195
76.81230000000001
76.8131
76.82190000000001
76.8387
76.86350000000002
76.89630000000001
76.9371
76.9859
77.0427
77.10750000000002
77.1803
77.2611
77.3499
77.44669999999999
77.5515
77.6643
77.78510000000001
77.9139
78.0507
78.19550000000001
78.3483
78.5091
78.67790000000001
78.85470000000001
77.6359
77.59890000000001
77.5657
77.5363
77.51070000000001
77.4889
77.47090000000001
77.45670000000001
77.4463
77.4397
77.43690000000001
77.43790000000001
77.4427
77.4513
77.4637
77.4799
77.49990000000001
77.5237
77.55130000000001
77.5827
77.6179
77.65690000000001
77.6997
77.7463
77.7967
77.8509
77.9089
77.9707
78.03630000000001
78.1057
78.17890000000001
78.25590000000001
78.3367
78.4213
78.50970000000001
78.60190000000001
78.6979
78.7977
78.9013
79.0087
79.1199
79.2349
79.3537
5519.1377
